# CartPole mit PolicyGradient

In [27]:
import ptan

PTAN (Practical Deep Reinforcement Learning) ist eine Python-Bibliothek, die beim Training von künstlichen neuronalen Netzen für Verstärkungslernen hilft. Es stellt verschiedene nützliche Funktionen und Tools zur Verfügung, um das Training und die Auswertung von Reinforcement-Learning-Modellen zu erleichtern.

In [28]:
import gym
import argparse
import numpy as np

In [29]:
from tensorboardX import SummaryWriter

Der SummaryWriter ermöglicht es, Summaries für TensorFlow-Graphen und -Variablen zu erstellen und in ein  bestimmtes Verzeichnis zu schreiben. Diese Summaries können dann von einem Tool wie TensorBoard geladen und visualisiert werden.

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [31]:
GAMMA = 0.99
LEARNING_RATE = 0.001
ENTROPY_BETA = 0.01
BATCH_SIZE = 8
REWARD_STEPS = 10

In [32]:
class PGN(nn.Module):
    def __init__(self, input_size, n_actions):
        super(PGN, self).__init__()
        # super ist der Aufruf der Init der Superklasse, hier nn.Module
        
        self.net = nn.Sequential(
            nn.Linear(input_size, 128), # Linear ist wie Dense aus Tensorflow
            nn.ReLU(),
            nn.Linear(128, n_actions)
        )
        
    def forward(self, x):
        return self.net(x)

# Main

In [33]:
parser = argparse.ArgumentParser()
parser.add_argument("--baseline", default=False, action="store_true", help="Enable mean baseline")

_StoreTrueAction(option_strings=['--baseline'], dest='baseline', nargs=0, const=True, default=False, type=None, choices=None, required=False, help='Enable mean baseline', metavar=None)

Mit der Methode add_argument() werden dem Parser Argumente hinzugefügt. Im gegebenen Beispiel wird das Argument `"--baseline"` definiert. Durch das Setzen von `"default=False"` wird festgelegt, dass dieses Argument standardmäßig den Wert False hat, falls es nicht in der Kommandozeile angegeben wird. Mit `"action='store_true'"` wird festgelegt, dass das Vorhandensein des Argumentes `'--baseline'` dazu führt, dass der Wert auf `True` gesetzt wird.
    
Die zusätzliche Angabe von `"help"` gibt eine Beschreibung des Arguments, die in der Hilfe ausgegeben wird, wenn der Benutzer die Option --help verwendet.

In [34]:
args = parser.parse_args(args=[])

Wenn wir das Beispiel ausführen und als Kommandozeilenargument `"--baseline"` angeben, wird `args.baseline` den Wert `True` haben. Andernfalls wird `args.baseline` den Standardwert False haben.

In [35]:
env = gym.make("CartPole-v0")

In [36]:
writer = SummaryWriter(comment="-cartpole-pg" + "-baseline=%s" % args.baseline)

In [37]:
net = PGN(env.observation_space.shape[0], env.action_space.n)
net

PGN(
  (net): Sequential(
    (0): Linear(in_features=4, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=2, bias=True)
  )
)

In [38]:
agent = ptan.agent.PolicyAgent(net, preprocessor=ptan.agent.float32_preprocessor, apply_softmax=True)

### PolicyAgent
Der PolicyAgent ist ein Schlüsselelement im Verstärkungslernen, da er die Strategie des Agenten definiert, dh wie der Agent Entscheidungen trifft, welche Aktionen er ausführt.

Der PolicyAgent wird verwendet, um die Richtlinie oder Strategie des neuronalen Netzes zu steuern. Das neuronale Netz (net) wird als Argument übergeben, um die Vorhersagen zu erhalten. Der preprocessor-Parameter bestimmt, wie die Eingabedaten vor der Verwendung durch das neuronale Netzwerk vorverarbeitet werden. Mit `ptan.agent.float32_preprocessor` wird eine Funktion angegeben, die die Eingabedaten in den Datentyp `float32` umwandelt. Die Option `apply_softmax=True` bestimmt, ob die Softmax-Funktion auf die Ausgabe des neuronalen Netzes angewendet werden soll, um die Wahrscheinlichkeiten für jede Aktion zu berechnen.

In [39]:
exp_source = ptan.experience.ExperienceSourceFirstLast(env, agent, gamma=GAMMA, steps_count=REWARD_STEPS)

### Experience Buffer
Die ExperienceSourceFirstLast-Klasse speichert eine festgelegte Anzahl von Schritten (`steps_count`) der Erfahrung des Agents (Zustände, Aktionen, Belohnungen, nächste Zustände) in einem internen Puffer. Bei jedem Schritt des Agenten wird eine neue Erfahrung in den Puffer geschrieben, und wenn die Anzahl der gespeicherten Schritte die festgelegte Puffergröße überschreitet, werden die ältesten Erfahrungen entfernt.

Während der Trainingsschleife werden die Erfahrungen aus dem Puffer genommen und verwendet, um das Netzwerk zu trainieren. Der Puffer hilft dabei, Erfahrungen zu speichern und in einer zufälligen Reihenfolge auf die Erfahrungen zuzugreifen, um das Training zu diversifizieren und das Wiederholen von Erfahrungen zu vermeiden.

Der Puffer ermöglicht es dem Agenten, frühere Erfahrungen wiederzuverwenden und von ihnen zu lernen, was zur Verbesserung der Entscheidungsfindung führen kann. Durch das Speichern von Erfahrungen über mehrere Schritte können auch zeitliche Zusammenhänge und kumulative Belohnungen im Training berücksichtigt werden.

Die Befüllung des `exp_source` Objekts findet statt, wenn es initialisiert wird. In der Zeile `exp_source = ptan.experience.ExperienceSourceFirstLast(env, agent, gamma=GAMMA, steps_count=REWARD_STEPS)` wird eine Episode von Erfahrungen generiert, indem der Agent agent mit der Umgebung env interagiert. Das Objekt `exp_source` speichert diese generierten Erfahrungen für den weiteren Gebrauch in der Trainingsschleife.

Die `exp_source`-Instanz, genauer gesagt die `ExperienceSourceFirstLast`-Klasse, speichert die einzelnen Erfahrungen des Agenten (Zustände, Aktionen, Belohnungen usw.) in einem Puffer. Dabei werden die zuvor verwendeten Erfahrungen mit jedem neuen Schritt aktualisiert, sodass ältere Erfahrungen schrittweise verfallen und aus dem Puffer entfernt werden.

Die `pop_total_rewards()`-Methode wird verwendet, um die Gesamtbelohnungen der abgeschlossenen Episoden aus `exp_source` zu extrahieren und zurückzugeben. Dies ermöglicht die Berechnung von Durchschnittsbelohnungen oder anderen Messwerten, um den Fortschritt des Trainings zu verfolgen. Die Methode hat jedoch keinen direkten Einfluss auf den Inhalt des Puffers in exp_source.

In [40]:
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)

### Adam Optimizer

Der Adam-Optimizer ist ein Optimierungsalgorithmus, der häufig in neuronalen Netzwerken verwendet wird. Er kombiniert Ideen aus dem Stochastic Gradient Descent (SGD) und dem RMSprop-Algorithmus.

Der Adam-Algorithmus berechnet adaptive Lernraten für jeden Parameter des neuronalen Netzwerks. Der Name `Adam` steht für "Adaptive Moment Estimation". Hier ist der grundlegende Ablauf des Adam-Optimierers:
1. Initialisierung der Parameter: Der Optimierer speichert zwei Zustandsvariablen `m` und `v`, die den ersten und den zweiten Moment des Gradienten schätzen.
2. Berechnung des Gradienten: Der Gradient der Verlustfunktion in Bezug auf die Parameter des Netzwerks wird berechnet.
3. Aktualisierung der Zustände `m` und `v`: Die Zustandsvariablen `m` und `v` werden aktualisiert, um die Momente des Gradienten zu schätzen. Das geschieht durch die Verwendung von Exponential Moving Averages der Gradienten und ihren Quadraten.
4. Berechnung des Bias-korrigierten ersten und zweiten Moments: Da die Schätzer `m` und `v` zu Beginn des Algorithmus stark abgeschwächt sind, führt der Adam-Algorithmus eine Korrektur durch, um den Bias in den Anfangswerten zu beheben.
5. Aktualisierung der Parameter: Die Parameter des Netzwerks werden mit Hilfe der berechneten adaptive Lernraten aktualisiert.

Der Adam-Optimizer hat sich als effizienter und robuster in der Praxis erwiesen als der SGD und der RMSprop-Algorithmus, da er adaptive Lernraten verwendet und die Momente des Gradienten schätzt. Dadurch kann er schneller konvergieren und bessere Generalisierungseigenschaften aufweisen.

#### Momente

In Bezug auf den Adam-Optimizer bezieht sich der Begriff "Moment" auf statistische Maße, die verwendet werden, um den Verlauf des Gradienten über mehrere Iterationen hinweg zu schätzen. Der Adam-Algorithmus verwendet zwei Momente - das erste und das zweite Moment des Gradienten.

Das erste Moment, oft als "mittlerer Gradient" bezeichnet, ist eine Schätzung des Durchschnitts des Gradients. Es wird durch die exponentielle Glättung der vergangenen Gradientschätzungen berechnet.

Das zweite Moment, oft als "variable Veränderung" bezeichnet, ist eine Schätzung des Durchschnitts der quadrierten Gradientschwankungen. Es wird auch durch die exponentielle Glättung der vergangenen Gradientschätzungen berechnet.

Diese beiden Momente werden verwendet, um die adaptiven Lernraten für die Aktualisierung der Parameter des neuronalen Netzwerks zu berechnen. Durch die Verwendung von Momenten kann der Adam-Optimizer Informationen über die Richtung und Stärke des Gradienten erhalten und so eine effektive Anpassung der Lernraten ermöglichen. Dies trägt zu einer effizienteren Optimierung des neuronalen Netzwerks bei.

In [41]:
total_rewards = []
step_rewards = []
step_idx = 0
done_episodes = 0
reward_sum = 0.
batch_states, batch_actions, batch_scales = [], [], []

In [42]:
import sys

In [43]:
# Beginn der Trainingsschleife
for step_idx, exp in enumerate(exp_source):
    reward_sum += exp.reward
    baseline = reward_sum / (step_idx + 1)
    writer.add_scalar("baseline", baseline, step_idx)
    batch_states.append(exp.state)
    batch_actions.append(int(exp.action))
    if args.baseline:
        batch_scales.append(exp.reward - baseline)
    else:
        batch_scales.append(exp.reward)
        
        
    # handle new rewards
    new_rewards = exp_source.pop_total_rewards() # total_rewards ist eine Liste von Rewards
    if new_rewards:
        done_episodes += 1
        reward = new_rewards[0]
        total_rewards.append(reward)
        mean_rewards = float(np.mean(total_rewards[-100:])) # Durchschnitt der letzten 100 Rewards
        print("%d: reward: %6.2f, mean_100: %6.2f, episodes: %d" % (
            step_idx, reward, mean_rewards, done_episodes))
        writer.add_scalar("reward", reward, step_idx)
        writer.add_scalar("reward_100", mean_rewards, step_idx)
        writer.add_scalar("episodes", done_episodes, step_idx)
        if mean_rewards > 195:
            print("Solved in %d steps and %d episodes!" % (step_idx, done_episodes))
            break

    if len(batch_states) < BATCH_SIZE:
            continue
        
    states_v = torch.FloatTensor(batch_states)
    batch_actions_t = torch.LongTensor(batch_actions)
    batch_scale_v = torch.FloatTensor(batch_scales)

    optimizer.zero_grad() # Gradienten zu 0 setzen
    logits_v = net(states_v) # Berechnung der Rohausgaben des Models
    log_prob_v = F.log_softmax(logits_v, dim=1) # logarithmierte Wahrscheinlichkeiten berechnen
    
    # Berechnung der Policy Loss
    log_p_a_v = log_prob_v[range(BATCH_SIZE), batch_actions_t] 
    log_prob_actions_v = batch_scale_v * log_p_a_v
    loss_policy_v = -log_prob_actions_v.mean()
    
    # Rückwertsdurchgang
    loss_policy_v.backward(retain_graph=True)
    grads = np.concatenate([p.grad.data.numpy().flatten()
                            for p in net.parameters()
                            if p.grad is not None])
    
    # Berechnung der Entropy Loss
    prob_v = F.softmax(logits_v, dim=1)
    entropy_v = -(prob_v * log_prob_v).sum(dim=1).mean()
    entropy_loss_v = -ENTROPY_BETA * entropy_v
    entropy_loss_v.backward()
    optimizer.step()

    # Berechnung des Gesamt Loss
    loss_v = loss_policy_v + entropy_loss_v

    # calc KL-div
    new_logits_v = net(states_v)
    new_prob_v = F.softmax(new_logits_v, dim=1)
    kl_div_v = -((new_prob_v / prob_v).log() * prob_v).sum(dim=1).mean()
    writer.add_scalar("kl", kl_div_v.item(), step_idx)

    writer.add_scalar("baseline", baseline, step_idx)
    writer.add_scalar("entropy", entropy_v.item(), step_idx)
    writer.add_scalar("batch_scales", np.mean(batch_scales), step_idx)
    writer.add_scalar("loss_entropy", entropy_loss_v.item(), step_idx)
    writer.add_scalar("loss_policy", loss_policy_v.item(), step_idx)
    writer.add_scalar("loss_total", loss_v.item(), step_idx)

    g_l2 = np.sqrt(np.mean(np.square(grads)))
    g_max = np.max(np.abs(grads))
    writer.add_scalar("grad_l2", g_l2, step_idx)
    writer.add_scalar("grad_max", g_max, step_idx)
    writer.add_scalar("grad_var", np.var(grads), step_idx)

    batch_states.clear()
    batch_actions.clear()
    batch_scales.clear()

writer.close()

16: reward:  16.00, mean_100:  16.00, episodes: 1
31: reward:  15.00, mean_100:  15.50, episodes: 2
54: reward:  23.00, mean_100:  18.00, episodes: 3
70: reward:  16.00, mean_100:  17.50, episodes: 4
84: reward:  14.00, mean_100:  16.80, episodes: 5
98: reward:  14.00, mean_100:  16.33, episodes: 6
113: reward:  15.00, mean_100:  16.14, episodes: 7
133: reward:  20.00, mean_100:  16.62, episodes: 8
148: reward:  15.00, mean_100:  16.44, episodes: 9
171: reward:  23.00, mean_100:  17.10, episodes: 10
212: reward:  41.00, mean_100:  19.27, episodes: 11
234: reward:  22.00, mean_100:  19.50, episodes: 12
244: reward:  10.00, mean_100:  18.77, episodes: 13
258: reward:  14.00, mean_100:  18.43, episodes: 14
275: reward:  17.00, mean_100:  18.33, episodes: 15
289: reward:  14.00, mean_100:  18.06, episodes: 16
390: reward: 101.00, mean_100:  22.94, episodes: 17
412: reward:  22.00, mean_100:  22.89, episodes: 18
426: reward:  14.00, mean_100:  22.42, episodes: 19
448: reward:  22.00, mean_1

- *Policy-Loss*: Der Policy-Loss dient dazu, das Netzwerk zu trainieren, eine bessere Politik (Strategie) zu lernen, die zu höheren Belohnungen führt. Durch die Maximierung des Policy-Losses werden gute Aktionen, die höhere Belohnungen erzielen, bevorzugt. In diesem Sinne optimiert der Policy-Loss die Richtung und Stärke der Aktionen des Agenten, um die Belohnung zu maximieren. Der Policy-Loss fördert somit das Lernen, welche Aktionen gute Ergebnisse erzeugen und verstärkt diese Aktionen weiter.

- ### Entropy-Loss
Der Entropy-Loss dient dazu, die Exploration des Agenten zu fördern und die Diversität der Aktionen zu erhöhen. Indem der Entropy-Loss in den Gesamt-Loss einbezogen wird, wird der Grad der Unsicherheit in der Aktionserzeugung maximiert. Dies hilft, die Exploration des Agenten zu erhöhen und sicherzustellen, dass das Netzwerk verschiedene Aktionen ausprobiert und nicht in einer vorzeitigen Konvergenz auf eine einzelne Aktion stecken bleibt. Der Entropy-Loss trägt dazu bei, die Wahrscheinlichkeitsverteilung der Aktionen zu regulieren und die Politik zu diversifizieren.

Der Policy-Loss und der Entropy-Loss werden oft kombiniert, um eine geeignete Gleichgewicht zwischen der Exploration und der Ausbeutung von guten Aktionen zu finden. Der Entropy-Loss stellt sicher, dass der Agent nicht in einer lokalen Optima stecken bleibt und ermöglicht die Exploration von neuen Aktionen, während der Policy-Loss die Richtung der Aktionen optimiert, um die Belohnungen zu maximieren.


- ### Summarywriter
Bevor das Training beginnt, erstellt man eine Instanz des SummaryWriter und gibt ihm den Pfad zum Verzeichnis an, in dem die Summaries gespeichert werden sollen. Während des Trainings oder der Evaluation können dann verschiedene Arten von Summaries, wie z.B. Skalare (z.B. Verlust oder Genauigkeit), Histogramme oder Bilder, mit Hilfe des SummaryWriters erstellt und regelmäßig in das angegebene Verzeichnis geschrieben werden.

- ### Neue Belohnungen
Neue Belohnungen: Wenn neue Belohnungen verfügbar sind, werden sie verarbeitet, indem sie zur Gesamtbelohnungsliste hinzugefügt werden und der Durchschnitt der letzten 100 Belohnungen berechnet wird. Diese Werte werden zur Tensorboard-Zusammenfassung hinzugefügt. Wenn der Durchschnitt der letzten 100 Belohnungen über 195 liegt, wird das Training als erfolgreich abgeschlossen angesehen und die Schleife wird beendet.

- ### Batch-Größe
Batch-Größe: Wenn genügend Erfahrungen gesammelt wurden, wird der nächste Schritt ausgeführt. Der aktuelle Batch von Zuständen, Aktionen und Skalierungen wird in Tensoren umgerechnet. 

- ### Vorwärtsdurchgang
Der Vorwärtsdurchgang des neuronalen Netzwerks wird durchgeführt und die log_softmax-Werte werden berechnet. Es wird auch der log_likelihood-Wert für die ausgewählten Aktionen berechnet.

- ### Richtungsrückgabe
Der Rückwärtsdurchgang wird durchgeführt und die Gradienten der Netzwerkparameter werden berechnet. Außerdem werden verschiedene Loss-Werte, wie der Policy-Loss und der Entropy-Loss, berechnet und zur Tensorboard-Zusammenfassung hinzugefügt.

- ### KL-Divergenz
Die KL-Divergenz zwischen den alten und den neuen Wahrscheinlichkeitsverteilungen wird berechnet und zur Tensorboard-Zusammenfassung hinzugefügt.